In [ ]:
# -*- coding: utf-8 -*-
import os 
import numpy as np
import matplotlib.pyplot as plt
import glob
from tkinter import filedialog as fd
import time
import datetime
from scipy.signal import find_peaks
import math 
from sklearn.linear_model import LinearRegression

###############################################################################
########## Code Body ##########################################################
###############################################################################
    
    
workingPath = fd.askdirectory()
os.chdir(workingPath)

if os.path.isfile("SynthesisFile.txt"):
    os.remove("SynthesisFile.txt")
    
for fileName in sorted(glob.glob("*.IP")):
    st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')  
                 
    #Read the CSV that contains all the data
    currentFile = open(fileName,'r')
    fileLines = currentFile.readlines() 
    fileNameOut = fileName.rstrip(".txt")
    currentFile.close()
    print(st)
    print("Processing: " + fileNameOut)
        
    Field = np.array([])
    hallVolt = np.array([])
    gate_voltage = np.array([])
    theta_M = np.array([])
    sin_theta = np.array([])
    cos_theta = np.array([])
    channel_voltage = np.array([])
    voltage_Counter = 1
    fit = np.array([])
    v_hall = np.array([])

    for lineIndex in range(len(fileLines)):
        currentLine = fileLines[lineIndex]
        dataSegment = currentLine.split()
        Field = np.append(Field, float(dataSegment[9])) #This is milli Tesla
        hallVolt = np.append(hallVolt, float(dataSegment[5])) #This is milli Volt
        channel_voltage = np.append(channel_voltage, float(dataSegment[2])) #This is Volt
        VG = float(dataSegment[6]) - float(dataSegment[2])/2
        gate_voltage = np.append(gate_voltage, np.round(VG, 2)) #This is Volt
        if lineIndex > 1 and VG != gate_voltage[lineIndex-1]:
            voltage_Counter = voltage_Counter + 1
            
    HV = hallVolt-((max(hallVolt))+min(hallVolt))/2
    # Split up field array as well ()  ##this line is written by my mentor
    HVN = HV/((max(HV)-min(HV))/2)
    
    for index in range(len(HVN)):
        if HVN[index] < 0:
            theta_M = np.append(theta_M, math.acos(HVN[index])-np.pi)
        else:
            theta_M = np.append(theta_M, math.acos(HVN[index]))

    for angle in range(len(theta_M)):
        sin_theta = np.append(sin_theta, math.sin(theta_M[angle]))
        cos_theta = np.append(cos_theta, math.cos(theta_M[angle]))

    #From here on issues in code
    x = sin_theta ** 2
    RHS = abs(Field * ((math.cos(86*np.pi/180)/cos_theta) - (math.sin(86*np.pi/180)/sin_theta)))
    array = np.column_stack([x, RHS, gate_voltage])
    
    dup = np.unique(gate_voltage)
    for i in dup:
        xFit = np.array([])
        RHSFit = np.array([])
        x_new = []
        RHS_new = []
        for a,b,c in array:
            if c == i: # change specific gate_voltage value
                x_new.append(a)
                RHS_new.append(b)
        x_new = np.array(x_new)
        RHS_new = np.array(RHS_new)
        for index in range(len(x_new)):
            if x_new[index] > 0.2 and x_new[index] < 0.7:
                xFit = np.append(xFit, x_new[index])
                RHSFit = np.append(RHSFit, RHS_new[index])

        model = LinearRegression().fit(xFit.reshape(-1,1), RHSFit)
        regression_line = xFit*model.coef_ + model.intercept_

        m = (((np.mean(xFit) * np.mean(RHSFit)) - np.mean(xFit*RHSFit)) / (np.mean(xFit)**2 -np.mean(xFit*RHSFit)))
        b = np.mean(RHSFit) - m*np.mean(xFit)
        regression_line = xFit*m + b

        print(f'======================= {i} ============================')
        plt.figure(1)
        plt.scatter(xFit, RHSFit, c = 'green')
        plt.plot(xFit, regression_line, c = 'r')
        figureName = '04. RHS vs LHS.png'
        plt.title("||µ0_H|| (cos\u03B8_H/cos\u03B8_M - sin\u03B8_H/sin\u03B8_M vs. sin²\u03B8M")
        plt.xlabel("sin²\u03B8M [arb. unit]")
        plt.ylabel("||\u03BC H|| (cos\u03B8_H/cos\u03B8_M - sin\u03B8_H/sin\u03B8_M [mT]")
        plt.savefig(os.path.join(workingPath, figureName), dpi = 200, format = 'png')
        plt.show()
        plt.close(1)
        

In [ ]:
# -*- coding: utf-8 -*-
import os 
import numpy as np
import matplotlib.pyplot as plt
import glob
from tkinter import filedialog as fd
import time
import datetime
from scipy.signal import find_peaks
import math 
from sklearn.linear_model import LinearRegression

###############################################################################
########## Code Body ##########################################################
###############################################################################
    
    
workingPath = fd.askdirectory()
os.chdir(workingPath)

if os.path.isfile("SynthesisFile.txt"):
    os.remove("SynthesisFile.txt")
    
for fileName in sorted(glob.glob("*.IP")):
    st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')  
                 
    #Read the CSV that contains all the data
    currentFile = open(fileName,'r')
    fileLines = currentFile.readlines() 
    fileNameOut = fileName.rstrip(".txt")
    currentFile.close()
    print(st)
    print("Processing: " + fileNameOut)
        
    Field = np.array([])
    hallVolt = np.array([])
    gate_voltage = np.array([])
    theta_M = np.array([])
    sin_theta = np.array([])
    cos_theta = np.array([])
    channel_voltage = np.array([])
    voltage_Counter = 1
    fit = np.array([])
    v_hall = np.array([])

    for lineIndex in range(len(fileLines)):
        currentLine = fileLines[lineIndex]
        dataSegment = currentLine.split()
        Field = np.append(Field, float(dataSegment[9])) #This is milli Tesla
        hallVolt = np.append(hallVolt, float(dataSegment[5])) #This is milli Volt
        channel_voltage = np.append(channel_voltage, float(dataSegment[2])) #This is Volt
        VG = float(dataSegment[6]) - float(dataSegment[2])/2
        gate_voltage = np.append(gate_voltage, np.round(VG, 2)) #This is Volt
        if lineIndex > 1 and VG != gate_voltage[lineIndex-1]:
            voltage_Counter = voltage_Counter + 1
            
    HV = hallVolt-((max(hallVolt))+min(hallVolt))/2
    array = np.column_stack([Field, HV, gate_voltage])
    np.savetxt('test.txt', array, delimiter=' ')

In [249]:
import pandas as pd
import numpy as np
df = pd.read_csv('test.txt', delim_whitespace=True, header=None)
df.columns=['Field','hall_voltage','gate_voltage']
unique = np.unique(df.gate_voltage)
df_new = df.sort_values(by=['gate_voltage']).reset_index(drop=True)
df_new.set_index('gate_voltage')
grouped = df_new.groupby('gate_voltage')
gp=[]
for name,group in grouped:
    gp.append(group)
    print(name)

#gp[0]
#gp[1]
#gp[2]
#gp[3]
#gp[4]

-3.0
-1.5
-0.0
1.5
3.0


,Field,hall_voltage,gate_voltage
0,-1025.735497,-11.101938,-3.0
1,-419.038355,-35.649571,-3.0
2,-413.659483,-36.519985,-3.0
3,-408.382267,-37.110791,-3.0
4,-403.213739,-38.095647,-3.0
...,...,...,...
796,-384.829640,-39.531525,-3.0
797,-395.666063,-38.279445,-3.0
798,1012.922287,10.926281,-3.0
799,1022.968769,11.054813,-3.0


In [247]:
# df_new=[]
# for item in unique:
#     gv=df.loc[df['gate_voltage']==item].reset_index(drop=True)
#     np.savetxt('gv_sep'+str(i)+'.txt', gv, delimiter=' ')


In [245]:
# a = len(df_new)
# n=[]
# for j in range(a):
#     new = pd.concat([df_new[j]],axis=1)
#     n.append(new)
# n

In [ ]:
# g_0=df.loc[df['gate_voltage']==0].reset_index(drop=True)
# g_m3=df.loc[df['gate_voltage']==-3].reset_index(drop=True)
# g_m1P5=df.loc[df['gate_voltage']==-1.5].reset_index(drop=True)
# g_3=df.loc[df['gate_voltage']==3].reset_index(drop=True)
# g_1P5=df.loc[df['gate_voltage']==1.5].reset_index(drop=True)

In [ ]:
# df_new = pd.concat([g_0,g_1P5,g_m1P5,g_3,g_m3],axis=1)

In [ ]:
# np.savetxt('gv_seperated.txt', df_new, delimiter=' ')